
#  Apply RNN Algorithm on Dataset

#  To Classify the Speech as Hate or Not

In [25]:
import pandas as pd

In [26]:
#Reading Dataset with Display of 4 Values

# 1000 Instances in labeled dataset

# df DataFrame is a 2-dimensional labeled data structure with
# columns of potentially different types.

df = pd.read_csv('train.csv')


df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [27]:
# Use variables to X (label : Sentence), Y (label : positive or negative)

x=df['tweet']
y=df['label']

In [28]:
from sklearn.model_selection import train_test_split

# Spilt data in Train & Test variables 20% train and 80% test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [29]:
print("shape of X_train: ",X_train.shape)
print("shape of X_test: ",X_test.shape)

print("shape of y_train: ",y_train.shape)
print("shape of y_test: ",y_test.shape)

shape of X_train:  (25569,)
shape of X_test:  (6393,)
shape of y_train:  (25569,)
shape of y_test:  (6393,)


In [30]:
# Print Train Data : Sentence with Comment LAbel(negative 0/ Postive 1)

print(X_train[6]) 
print(y_train[6])

 @user camping tomorrow @user @user @user @user @user @user @user dannyâ¦
0


In [31]:
# we will use the Tokenizer class : create a word-to-index dictionary. In the word-to-index dictionary,
# 
from keras.preprocessing.text import Tokenizer

# Top 1000 most frequent words
tokenizer = Tokenizer(num_words=1000,lower=True)
# Fitting Data
tokenizer.fit_on_texts(X_train)


# Sequencing the textual data (Converting into number)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)


# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1


In [32]:
print(X_train[6]) 
print(y_train[6])

[360, 427]
0


In [33]:
# from keras.preprocessing.sequence import pad_sequences 
# This library is out of support for new versions (Remember)

from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 100  #We set the maximum size of each list to 100.

# pad_sequences is used to ensure that all sequences in a list have the same length.

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

##checking dataset locations
print(X_train[3, :]) #the resulting feature vector contains mostly zeros, since you have a fairly short sentence.

[ 12  89  48  20 281 281  89   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [34]:
from keras.utils import to_categorical
num_classes = 2

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(y_train.shape)
print(y_train[0])
#print(len(y_train[0]))

(25569, 2)
[1. 0.]


In [35]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,SimpleRNN
from keras import optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# import numpy as np

# (samples, timesteps, features)
# (batch_size, timesteps, input_dim)

X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)  #(750, 100, 1)

X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)

# Algo Requires 3d Data

(25569, 100, 1)
(6393, 100, 1)


In [36]:

num_classes=2

def vanilla_rnn():
    model = Sequential()
    #model.add(SimpleRNN(50 neurons, input_shape = (X_train.shape[1], X_train.shape[2],), return_sequences = False))
    model.add(SimpleRNN(50, input_shape = (maxlen,1), return_sequences = False))
    model.add(Dense(num_classes))
    model.add(Activation('softmax')) # softmax converts vector of no. into vector of prob.
    # Calculate Prob of both Classes and class higher prob sentence 
    # will belong to that class
    model.summary()
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy']) # Calculates how often predictions equal labels.


    
    return model

In [37]:
from keras.wrappers.scikit_learn import KerasClassifier

# An epoch means training the neural
# network with all the training data for one cycle
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 2, batch_size = 50)
model.fit(X_train, y_train)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 50)                2600      
                                                                 
 dense_1 (Dense)             (None, 2)                 102       
                                                                 
 activation_1 (Activation)   (None, 2)                 0         
                                                                 
Total params: 2,702
Trainable params: 2,702
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2


C:\Users\lenevo\AppData\Local\Temp\ipykernel_9672\2998331466.py:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = vanilla_rnn, epochs = 2, batch_size = 50)
C:\Users\lenevo\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


512/512 [==============================] - 7s 12ms/step - loss: 0.2641 - accuracy: 0.9267
Epoch 2/2
512/512 [==============================] - 6s 12ms/step - loss: 0.2565 - accuracy: 0.9293


# Model Accuracy


In [38]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

# Accuracy predition

print("Percentage : ", 100*accuracy_score(y_pred, y_test_))

200/200 [==============================] - 1s 5ms/step
Percentage :  93.21132488659471


# Testing model on New input

In [39]:

a= ['good ']
a = tokenizer.texts_to_sequences(a)
a=np.array(a)
# Preparing data to send to model and formating
# it according to model
a = pad_sequences(a, padding='post', maxlen=maxlen)

a = a.reshape((a.shape[0], a.shape[1], 1))
print(a.shape)

prediction = model.predict(np.array(a))
print(prediction) 
type(prediction)

if prediction == [0]:
    # 1 means speech is Hate
    print("Speech is Hate")

if prediction == [1]:
    # 0 means speech is Not Hate 
    print("Speech is Not Hate")


(1, 100, 1)
1/1 [==============================] - 0s 24ms/step
[0]
Speech is Hate
